In [7]:
from allennlp.predictors.predictor import Predictor
import pickle
import tqdm
import pandas as pd

In [2]:
import allennlp_models.tagging

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")

C:\Users\aubin\anaconda3\envs\nlp-new\lib\site-packages\allennlp\tango\__init__.py:17: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  warnings.warn(
2022-01-18 09:28:30,816 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-01-18 09:28:31,536 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz is up-to-date
2022-01-18 09:28:31,537 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz from cache at C:\Users\aubin\.allennlp\cache\038f918d294bd1a45e3709dfb22af5277b0be8677f750a85748c39979ce0e549.b897bfe76a04a5f70d6e88762a4d819b4b8b90e45b31b8314e0a6a9630d3f213
2022-01-18 09:28:31,538 - INFO - allennlp.models.archival - extracting archive file C:\Users\aubin\.allennlp\cache\038f918d294bd1a45e3709dfb22af5277b0be8677f750a8

Neural Coref generates a cluster of similar words. The point now is to loop over all the docs and get these clusters.
Later on the point will be to link on of the elements in the cluster to an entity and that can lead to more relationships.

In [3]:
file = open("../stored_data/articles_cleaned.obj",'rb')
articles_cleaned = pickle.load(file)
file.close()

In [4]:
articles_corefed = {}

In [9]:
for idx, id in tqdm.tqdm(enumerate(list(articles_cleaned.keys()))):
    print((idx/len(list(articles_cleaned.keys())))*100)
    pred = predictor.predict(
        document=articles_cleaned[id]
    )

    clusters = pred['clusters']
    document = pred['document'] 

    articles_corefed[id] = [clusters, document]

0it [00:00, ?it/s]

0.0


0it [00:00, ?it/s]


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 50331648 bytes.

In [ ]:
filehandler = open("../stored_data/articles_corefed.obj","wb")
pickle.dump(articles_corefed, filehandler)
filehandler.close()

Sample code for how to make sense of the outputs of the neural coref.

In [ ]:
clusters = pred['clusters']
document = pred['document'] 

n = 0
doc = {}
for obj in document:    
    doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.    
    n = n+1

In [ ]:
clus_all = []
cluster = []
clus_one = {}
for i in range(0, len(clusters)):    
    one_cl = clusters[i]    
    for count in range(0, len(one_cl)):           
        obj = one_cl[count]        
        for num in range((obj[0]), (obj[1]+1)):            
            for n in doc:                
                if num == n:                 
                    cluster.append(doc[n]) 
    clus_all.append(cluster)       
    cluster = []     
print (clus_all) #And finally, this shows all coreferences